In [22]:
import pandas as pd
from langdetect import detect
import numpy as np
import matplotlib.pyplot as plt
import numpy

In [3]:
train = pd.read_csv("C:/Users/mideboer.EUROPE/Documents/GitHub/aml-mlops-workshop/outputs/raw_data/raw_subset_train.csv")
test = pd.read_csv("C:/Users/mideboer.EUROPE/Documents/GitHub/aml-mlops-workshop/outputs/raw_data/raw_subset_test.csv")

In [4]:
print(train)

Unnamed: 0                                               text  target
0            494  From: clldomps@cs.ruu.nl (Louis van Dompselaar...       1
1            276  From: bcash@crchh410.NoSubdomain.NoDomain (Bri...       3
2           1786  From: havardn@edb.tih.no (Haavard Nesse,o92a) ...       1
3            836  From: et@teal.csn.org (Eric H. Taylor) Subject...       2
4            781  From: kevin@rotag.mi.org (Kevin Darcy) Subject...       3
...          ...                                                ...     ...
1012         982  From: millernw@craft.camp.clarkson.edu (Neal M...       1
1013        1403  From: ad354@Freenet.carleton.ca (James Owens) ...       3
1014         961  From: sp1marse@lina (Marco Seirio) Subject: Su...       1
1015        1084  From: caldwell@facman.ohsu.edu (Larry Caldwell...       3
1016        1182  From: keith@cco.caltech.edu (Keith Allan Schne...       0

[1017 rows x 3 columns]


In [5]:
print(test)

Unnamed: 0                                               text  target
0          1263  From: lazio@astrosun.tn.cornell.edu (T. Joseph...       2
1            84  From: kmr4@po.CWRU.edu (Keith M. Ryan) Subject...       0
2           289  From: len@schur.math.nwu.edu (Len Evens) Subje...       2
3           371  From: wats@scicom.AlphaCDC.COM (Bruce Watson) ...       2
4          1272  From: thomsonal@cpva.saic.com Subject: Drag-fr...       2
..          ...                                                ...     ...
671        1329  From:  (Rashid) Subject: Re: Yet more Rushdie ...       0
672          24  From: daveb@pogo.wv.tek.com (Dave Butler) Subj...       3
673        1325  From: livesey@solntze.wpd.sgi.com (Jon Livesey...       0
674         444  From: hexham@acs.ucalgary.ca (Irving Hexham) S...       3
675         737  From: isaackuo@skippy.berkeley.edu (Isaac Kuo)...       3

[676 rows x 3 columns]


In [6]:
print(train.text.dtype)
print(test.text.dtype)
print(train.target.dtype)
print(test.target.dtype)

object
object
int64
int64


In [32]:
train.target.dtype == object

False

In [10]:
print(train.isnull().all())
print(test.isnull().all())

Unnamed: 0    False
text          False
target        False
dtype: bool
Unnamed: 0    False
text          False
target        False
dtype: bool


In [11]:
lang_train =  np.array([])
lang_test = np.array([])
for i in range(0, len(train)):
    lang_train =  np.append(lang_train,detect(train.text.values[i]))

for i in range(0, len(test)):
    lang_test = np.append(lang_test, detect(test.text.values[i]))

In [12]:
test2 = lang_train == 'en'
test1 = lang_test == 'en'
print(test2.all())
print(test1.all())

True
True


In [13]:
classes = train.target.value_counts(normalize=True)
train_mean = classes.mean()
print(train_mean)
train_std = classes.std()
print(train_std)
if train_std > 0.05:
    print("WARNING: CLasses might be imbalanced")

0.25
0.04287821802571039


In [14]:
classes = test.target.value_counts(normalize=True)
test_mean = classes.mean()
print(test_mean)
test_std = classes.std()
print(test_std)
if train_std > 0.05:
    print("WARNING: CLasses might be imbalanced")


0.25
0.04204932072056773


In [15]:
# test if text always start with "from"


In [16]:
import string

# make every thing lower case
train.text = train.text.apply(lambda x: x.lower())
test.text = test.text.apply(lambda x: x.lower())

# remove punctuation
translator = str.maketrans('', '', string.punctuation)
train.text = train.text.apply(
    lambda x: x.translate(translator))
test.text = test.text.apply(
    lambda x: x.translate(translator))

# remoce digits
train.text = train.text.apply(
    lambda x: x.translate(string.digits))
test.text = test.text.apply(
    lambda x: x.translate(string.digits))


In [31]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = stopwords.words('english')

clean_text_train = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_text_test = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mideboer.EUROPE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
from collections import Counter

vocab = Counter()

for text in clean_text_train:
    for word in text.split(' '):
        vocab[word.lower()] += 1

for text in clean_text_test:
    for word in text.split(' '):
        vocab[word.lower()] += 1


df = pd.DataFrame(list(vocab.items()), columns=['word', 'count'])
df1 = df.sort_values('count', ascending=False)
print(df1.head(50))


word  count
4             subject   1857
13              lines   1795
7        organization   1692
76              would   1576
21                one   1559
19             writes   1362
43            article   1116
84               dont   1027
506               god    969
1044           people    947
133              like    908
227              know    872
162             space    846
23              image    817
9          university    804
574              also    785
85              think    780
66    nntppostinghost    745
229               see    666
655               get    628
794              many    627
457               may    622
851                us    621
45                say    617
117             could    608
873            system    597
257              time    587
348              even    580
709              well    568
1034              use    565
887              good    562
48                 im    550
230               way    537
697              much    509
17

In [141]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text'].apply(lambda x: avg_word(x))
train[['text','avg_word']].head()

,text,avg_word
0,from lazioastrosuntncornelledu t joseph lazio ...,5.168490
1,from kmr4pocwruedu keith m ryan subject re 27 ...,4.815951
2,from lenschurmathnwuedu len evens subject re s...,4.930556
3,from watsscicomalphacdccom bruce watson subjec...,5.657534
4,from thomsonalcpvasaiccom subject dragfree sat...,4.900794


In [ ]:
test['avg_word'] = test['text'].apply(lambda x: avg_word(x))
test[['text','avg_word']].head()

In [144]:
print(train['avg_word'].mean())
print(test['avg_word'].mean())

5.163787187913941
5.177996244395777


In [ ]:
train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['text','stopwords']].head()

In [ ]:
test['stopwords'] = test['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
test[['text','stopwords']].head()

In [ ]:
print(train['stopwords'].mean())
print(test['stopwords'].mean())

In [ ]:
from textblob import TextBlob
train['sentiment'] = train['text'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['text','sentiment']].head()

In [ ]:
from textblob import TextBlob
test['sentiment'] = test['text'].apply(lambda x: TextBlob(x).sentiment[0] )
test[['text','sentiment']].head()

In [ ]:
print(train['sentiment'].mean())
print(test['sentiment'].mean())

In [64]:
from collections import Counter
df2= pd.DataFrame()
df1 = pd.DataFrame()

for classes in range(0,4):
    vocab = Counter()
    df_train = clean_text_train[train.target == classes]
    for text in df_train:
        for word in text.split(' '):
            vocab[word.lower()] += 1

    word1 = 'word' + str(classes)
    count = 'count' + str(classes)
    idf = 'idf' + str(classes)
    tfidf = 'tfidf' + str(classes)

    df = pd.DataFrame(list(vocab.items()), columns=[word1, count])

    for i,word in enumerate(df[word1]):
        df.loc[i, idf ] = np.log(df_train.shape[0]/(len(df_train[df_train.str.contains(word)]))) 

    array = df[count] * df[idf]
    df[tfidf] = array

    df1 = df.sort_values(tfidf, ascending=False, ignore_index=True)
    df2[word1] = df1[word1]
    df2[count] = df1[count]
    df2[idf]  =df1[idf]
    df2[tfidf] = df1[tfidf]

In [65]:
df2.head(20)

,word0,count0,idf0,tfidf0,word1,count1,idf1,tfidf1,word2,count2,idf2,tfidf2,word3,count3,idf3,tfidf3
0,god,384,0.950482,364.985125,jpeg,220,3.278859,721.348876,launch,206,1.599039,329.402076,judas,75,3.912023,293.401725
1,jesus,146,2.104975,307.326327,image,352,1.599216,562.924158,space,554,0.500427,277.236512,jesus,194,1.406497,272.860431
2,people,294,0.918394,270.007772,images,160,2.275556,364.089027,data,129,1.995921,257.473753,god,207,0.941609,194.912968
3,system,132,1.916923,253.033785,files,156,1.939084,302.497133,would,265,0.863692,228.878293,people,217,0.879477,190.846457
4,matthew,67,3.526361,236.266155,gif,106,2.344549,248.522225,satellite,94,2.327278,218.764104,objective,76,2.353878,178.894757
5,atheism,135,1.711071,230.994525,software,137,1.784934,244.535890,commercial,71,2.769110,196.606843,would,236,0.744440,175.687952
6,bible,121,1.888752,228.538960,data,110,2.180246,239.827086,shuttle,100,1.788281,178.828120,tyre,51,3.352407,170.972768
7,argument,129,1.758699,226.872120,file,171,1.400088,239.414994,program,102,1.689190,172.297411,greek,54,3.101093,167.459011
8,atheists,204,1.090244,222.409784,also,140,1.706462,238.904664,moon,107,1.565138,167.469729,bible,105,1.584745,166.398256
9,would,256,0.808832,207.060884,version,135,1.725510,232.943861,dont,118,1.396802,164.822676,morality,90,1.832581,164.932332
